In [1]:
set.seed(2000)
library(tidyverse)
library(tidymodels)
library(repr)
library(cowplot)
library(GGally)
library(ISLR)
library(themis)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

ERROR: Error in library(themis): there is no package called ‘themis’


In [ ]:
temp <- tempfile()
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip", temp)
pulsar_file <- unz(temp, "HTRU_2.csv")
pulsar <- read_csv(pulsar_file, col_names = FALSE)

In [ ]:
colnames(pulsar) <- c("mean_ip", "std_ip", "kurt_ip", "skew_ip", "mean_dm_snr", "std_dm_snr", "kurt_dm_snr", "skew_dm_snr", "class")

pulsar_mutate <- pulsar %>%
                mutate(class = as_factor(class))
pulsar_mutate

In [ ]:
# average of each column containing dbl values
pulsar_with_column_means <- pulsar %>%
                select(mean_ip:skew_dm_snr) %>%
                map_df(mean)
pulsar_with_column_means

In [ ]:
pulsar_split <- initial_split(pulsar_mutate, prop = 0.75, strata = class)
pulsar_train <- training(pulsar_split) 
pulsar_test <- testing(pulsar_split)

In [ ]:
options(repr.plot.height = 15, repr.plot.width = 15)
correlation <- ggpairs(pulsar_train, mapping = aes(color = class, alpha = 0.5))
correlation
# options(repr.plot.height = 10, repr.plot.width = 10)
### all predictor variables can be used because there seems to be a cut off for each variable regarding the areas that factor 0 and 1 take up more or less 

In [ ]:
pulsar_train_scaled <- pulsar_train %>% 
 mutate(mean_ip = scale(mean_ip, center = TRUE), 
        std_ip = scale(std_ip, center = TRUE), 
        kurt_ip = scale(kurt_ip, center = TRUE), 
        skew_ip = scale(skew_ip, center = TRUE), 
        mean_dm_snr = scale(mean_dm_snr, center = TRUE), 
        std_dm_snr = scale(std_dm_snr, center = TRUE), 
        kurt_dm_snr = scale(kurt_dm_snr, center = TRUE), 
        skew_dm_snr = scale(skew_dm_snr, center = TRUE))
pulsar_train_scaled

In [ ]:
sum(is.na(pulsar_train)) #checking for missing values in training data 

In [ ]:
#pulsar observation counts with 0's and 1's
count_train_pulsar <- pulsar_train %>%
    group_by(class) %>%
    summarize(n = n())
count_train_pulsar

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 13)

plot_mean_ip_dmsnr <- ggplot(pulsar_train_scaled, aes(x = mean_dm_snr, y =mean_ip , color = class)) +
                geom_point(alpha = 0.4) +
                labs(x = "Mean of the DM-SNR curve", y = "Mean integrated profile", color = "Class") +
                theme(text = element_text(size = 12)) +
                ggtitle("Mean integrated profile vs Mean of the DM-SNR curve") 

plot_skew_ip_dmsnr <- ggplot(pulsar_train_scaled, aes(x = skew_dm_snr, y= skew_ip , color = class)) +
                geom_point(alpha = 0.1) +
                labs(x = "Skewness of the DM-SNR curve", y = "Skewness integrated profile", color = "Class") +
                theme(text = element_text(size = 12)) +
                ggtitle("Skewness integrated profile vs Skewness of the DM-SNR curve") 

plot_kurt_ip_dmsnr <- ggplot(pulsar_train_scaled, aes(x = kurt_dm_snr, y= kurt_ip , color = class)) +
                geom_point(alpha = 0.4) +
                labs(x = "kurtosis of the DM-SNR curve", y = "kurtosis integrated profile", color = "Class") +
                theme(text = element_text(size = 12)) +
                ggtitle("Kurtosis integrated profile vs Kurtosis of the DM-SNR curve") 

plot_std_ip_dmsnr <- ggplot(pulsar_train_scaled, aes(x = kurt_dm_snr, y= kurt_ip , color = class)) +
                geom_point(alpha = 0.1) +
                labs(x = "Standard deviation of the DM-SNR curve", y = "Standard deviation integrated profile", color = "Class") +
                theme(text = element_text(size = 9)) +
                ggtitle("Standard deviation of integrated profile vs Standard deviation of the DM-SNR curve") 

plot_grid(plot_mean_ip_dmsnr, plot_skew_ip_dmsnr, plot_kurt_ip_dmsnr, plot_std_ip_dmsnr, ncol = 2, nrow = 2)

In [ ]:
###SCALED PLOTS ABOVE 

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 13)

plot_mean_ip_dmsnr <- ggplot(pulsar_train, aes(x = mean_dm_snr, y =mean_ip , color = class)) +
                geom_point(alpha = 0.4) +
                labs(x = "Mean of the DM-SNR curve", y = "Mean integrated profile", color = "Class") +
                theme(text = element_text(size = 12)) +
                ggtitle("Mean integrated profile vs Mean of the DM-SNR curve") 

plot_skew_ip_dmsnr <- ggplot(pulsar_train, aes(x = skew_dm_snr, y= skew_ip , color = class)) +
                geom_point(alpha = 0.1) +
                labs(x = "Skewness of the DM-SNR curve", y = "Skewness integrated profile", color = "Class") +
                theme(text = element_text(size = 12)) +
                ggtitle("Skewness integrated profile vs Skewness of the DM-SNR curve") 

plot_kurt_ip_dmsnr <- ggplot(pulsar_train, aes(x = kurt_dm_snr, y= kurt_ip , color = class)) +
                geom_point(alpha = 0.4) +
                labs(x = "kurtosis of the DM-SNR curve", y = "kurtosis integrated profile", color = "Class") +
                theme(text = element_text(size = 12)) +
                ggtitle("Kurtosis integrated profile vs Kurtosis of the DM-SNR curve") 

plot_std_ip_dmsnr <- ggplot(pulsar_train, aes(x = kurt_dm_snr, y= kurt_ip , color = class)) +
                geom_point(alpha = 0.1) +
                labs(x = "Standard deviation of the DM-SNR curve", y = "Standard deviation integrated profile", color = "Class") +
                theme(text = element_text(size = 9)) +
                ggtitle("Standard deviation of integrated profile vs Standard deviation of the DM-SNR curve") 

plot_grid(plot_mean_ip_dmsnr, plot_skew_ip_dmsnr, plot_kurt_ip_dmsnr, plot_std_ip_dmsnr, ncol = 2, nrow = 2)

In [ ]:
###UNSCALED PLOTS ABOVE

In [ ]:
### start data analysis = look for K 

In [ ]:
# ACCOUNTING FOR ALL 8 PREDICTOR VARIABLES 
# RECIPE USES TRAINING DATA 
pulsar_recipe <- recipe(class ~. , data = pulsar_train) %>%
  step_scale(all_predictors()) %>%
  step_center(all_predictors())

# MODEL SPEC TO DETERMINE K 
pulsar_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) %>% 
  set_engine("kknn") %>%
  set_mode("classification")

# USING CROSS-VALIDATION TO HELP US DETERMINE K 
# CHOSE V = 5 INSTEAD OF V=10 TO REDUCE RUN TIME 
pulsar_vfold <- vfold_cv(pulsar_train, v = 5, strata = class)


In [ ]:
# WE ARE LOOKING AT K VALUES BETWEEN 1-15 
k_vals <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

# GETTING RESULTS FOR THE RECIPE, MODEL SPEC, AND CROSS-VALIDATION FROM ABOVE 
pulsar_results <- workflow() %>%
  add_recipe(pulsar_recipe) %>% # recipe
  add_model(pulsar_spec) %>% # model spec with tune()
  tune_grid(resamples = pulsar_vfold, grid = k_vals) %>%
  collect_metrics() 

# GETTING ACCURACIES SO WE CAN PLOT AN ACCURACY VS K PLOT TO DETERMINE OUR IDEAL K 
accuracies <- pulsar_results %>%
  filter(.metric == "accuracy")

accuracies

# PLOTTING ACCURACY VS K 
accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") + 
  theme(text = element_text(size = 12))

accuracy_vs_k


In [ ]:
accuracy_for_7 <- accuracies %>%
        filter(neighbors == 7)
accuracy_for_7

In [ ]:
pulsar_final_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = 7) %>% 
  set_engine("kknn") %>%
  set_mode("classification")
                                      
pulsar_final_fit <- workflow() %>%
    add_recipe(pulsar_recipe) %>% 
    add_model(pulsar_final_spec) %>% 
    fit(data = pulsar_train)
                                      
prediction <- predict(pulsar_final_fit, pulsar_test) %>%
      bind_cols(pulsar_test)

prediction

test_set_accuracy <- prediction %>% # predictions object  
  metrics(truth = class, estimate = .pred_class) %>% # cat. variable
  filter(.metric == "accuracy") %>% # gives estimated accuracy %
  select(.estimate) %>% # for returning of only one value
  pull() # return only one value 

test_set_accuracy
                                      